In [1]:
import os
import sys
import re

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/spark/third-party-jars/spark-streaming-kafka-0-10_2.12-3.1.1.jar;/usr/local/spark/third-party-jars/spark-sql-kafka-0-10_2.12-3.1.1.jar pyspark-shell'

# Configure the environment
# if 'SPARK_HOME' not in os.environ:
# os.environ['SPARK_HOME'] = '/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7'
# Create a variable for our root path
# SPARK_HOME = os.environ['SPARK_HOME']
# print(SPARK_HOME)
#Set python location 
# sys.path.append("/usr/local/lib/python3.7/site-packages")
#Set python location 
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python")
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip")


from pyspark.sql import SparkSession, SQLContext
from pyspark import  SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# Spark session & context
conf = SparkConf()
conf.setMaster("local").setAppName("hello-world")
conf.set("spark.executor.memory", "1g")
conf.set("spark.executor.cores", "1")
conf.set("spark.cores.max", "4")
conf.set("spark.driver.memory",'1g')
conf.set("spark.driver.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.executor.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.sql.caseSensitive", "true")
conf.set("spark.ui.port", "4040")

sc = SparkContext(conf=conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
.container { 
    width:95% !important; 
}
</style>

# Preprocessing

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [4]:
#udf

# check if segment is not an empty segment (two vertex is the same)
@udf(returnType=BooleanType())
def is_not_the_same(from_latlon, to_latlon):
    return (from_latlon[0] != to_latlon[0]) | (from_latlon[1] != to_latlon[1])

In [5]:
routes = spark.read.csv('hdfs://namenode:8020/ola/static_data/routes.txt', header=True)
routes.select("route_id").show(100)

+---------+
| route_id|
+---------+
|  2-13139|
|  4-13139|
| 10-13139|
| 14-13139|
| 16-13139|
| 18-13139|
| 20-13139|
| 28-13139|
| 30-13139|
| 33-13139|
| 35-13139|
| 40-13139|
| 45-13139|
| 51-13139|
| 53-13139|
| 55-13139|
| 60-13139|
| 62-13139|
| 66-13139|
| 68-13139|
| 70-13139|
| 71-13139|
| 76-13139|
| 78-13139|
| 81-13139|
| 83-13139|
| 90-13139|
| 92-13139|
| 94-13139|
| 96-13139|
|102-13139|
|105-13139|
|106-13139|
|108-13139|
|110-13139|
|111-13139|
|115-13139|
|117-13139|
|120-13139|
|125-13139|
|127-13139|
|128-13139|
|130-13139|
|150-13139|
|152-13139|
|154-13139|
|155-13139|
|158-13139|
|161-13139|
|163-13139|
|164-13139|
|165-13139|
|166-13139|
|167-13139|
|169-13139|
|175-13139|
|176-13139|
|180-13139|
|183-13139|
|200-13139|
|201-13139|
|202-13139|
|204-13139|
|205-13139|
|206-13139|
|207-13139|
|209-13139|
|210-13139|
|211-13139|
|212-13139|
|217-13139|
|218-13139|
|222-13139|
|224-13139|
|230-13139|
|232-13139|
|233-13139|
|234-13139|
|236-13139|
|237-13139|
|239

In [6]:
trips = spark.read.csv('hdfs://namenode:8020/ola/static_data/trips.txt', header=True)
trips.show(1000, False)

+---------+-----------------------+--------------------------------+-------------+------------+--------+-------------+--------------+
|route_id |service_id             |trip_id                         |trip_headsign|direction_id|block_id|shape_id     |perm_trip_id  |
+---------+-----------------------+--------------------------------+-------------+------------+--------+-------------+--------------+
|611-13139|DEC20-D02CAR-1_Weekday |52088401-DEC20-D02CAR-1_Weekday |null         |1           |6110300 |6110028_DEC20|10611000280539|
|611-13139|DEC20-D02CAR-1_Weekday |52088402-DEC20-D02CAR-1_Weekday |null         |1           |6110100 |6110029_DEC20|10611000290605|
|611-13139|DEC20-D02CAR-1_Weekday |52088403-DEC20-D02CAR-1_Weekday |null         |1           |6110200 |6110029_DEC20|10611000290705|
|611-13139|DEC20-D02CAR-1_Weekday |52088404-DEC20-D02CAR-1_Weekday |null         |1           |6110300 |6110029_DEC20|10611000290801|
|611-13139|DEC20-D02CAR-1_Weekday |52088405-DEC20-D02CAR-1_Wee

In [21]:
shapes = spark.read.csv('hdfs://namenode:8020/ola/static_data/shapes.txt', header=True)
shapes.where("shape_id like '1800198_DEC20'").count()

717

In [10]:
routes_with_shapes = routes\
    .join(trips, "route_id")\
    .join(shapes, "shape_id")\
    .select("route_id", "shape_id", "shape_pt_lat", "shape_pt_lon", "shape_pt_sequence")\
    
routes_with_shapes.cache()
routes_with_shapes.show(10, False)

+---------+-------------+-------------+---------------+-----------------+
|route_id |shape_id     |shape_pt_lat |shape_pt_lon   |shape_pt_sequence|
+---------+-------------+-------------+---------------+-----------------+
|180-13139|1800196_DEC20|34.1001914421|-118.3255826782|10001            |
|180-13139|1800196_DEC20|34.1001875827|-118.3255872876|10002            |
|180-13139|1800196_DEC20|34.1001655287|-118.3256136273|10003            |
|180-13139|1800196_DEC20|34.1001104826|-118.3256464449|10004            |
|180-13139|1800196_DEC20|34.1000912298|-118.3256529763|10005            |
|180-13139|1800196_DEC20|34.0997972097|-118.3256518313|10006            |
|180-13139|1800196_DEC20|34.099799986 |-118.3266593125|10007            |
|180-13139|1800196_DEC20|34.1016300729|-118.3266598555|10008            |
|180-13139|1800196_DEC20|34.1016385394|-118.3253551031|10009            |
|180-13139|1800196_DEC20|34.1016385394|-118.3253551031|20001            |
+---------+-------------+-------------

In [31]:
route_shape_occurences = routes_with_shapes\
    .groupBy("route_id", "shape_id")\
    .count()\
    .orderBy("count")\

route_shape_occurences.show(10, False)

+---------+-------------+-----+
|route_id |shape_id     |count|
+---------+-------------+-----+
|161-13139|1610096_DEC20|100  |
|18-13139 |180139_DEC20 |168  |
|260-13139|2600265_DEC20|207  |
|211-13139|2110058_DEC20|211  |
|206-13139|2060174_DEC20|252  |
|180-13139|1800151_DEC20|261  |
|202-13139|2020050_DEC20|276  |
|180-13139|1800170_DEC20|285  |
|18-13139 |180110_DEC20 |286  |
|111-13139|1110336_DEC20|295  |
+---------+-------------+-----+
only showing top 10 rows



In [29]:
routes_with_most_occur_shape = route_shape_occurences\
    .withColumn('rn', row_number().over(Window.partitionBy('route_id').orderBy(col('count').desc())))\
    .where('rn = 1')\
    .select("route_id", "shape_id")\
    .join(shapes, "shape_id")\

routes_with_most_occur_shape.cache()
routes_with_most_occur_shape.show(10)
routes_with_most_occur_shape.count()

+-------------+---------+-------------+---------------+-----------------+
|     shape_id| route_id| shape_pt_lat|   shape_pt_lon|shape_pt_sequence|
+-------------+---------+-------------+---------------+-----------------+
|1020064_DEC20|102-13139| 33.950004218|-118.3928137447|            10001|
|1020064_DEC20|102-13139|33.9507939301| -118.392821075|            10002|
|1020064_DEC20|102-13139|33.9507951784|-118.3924376032|            10003|
|1020064_DEC20|102-13139|33.9494320666|-118.3924058284|            10004|
|1020064_DEC20|102-13139|33.9494294801| -118.392609587|            10005|
|1020064_DEC20|102-13139|33.9494266345|-118.3928083984|            10006|
|1020064_DEC20|102-13139|33.9488811994|-118.3928035327|            10007|
|1020064_DEC20|102-13139| 33.948881809|-118.3928694803|            10008|
|1020064_DEC20|102-13139|33.9488646177|-118.3939278131|            10009|
|1020064_DEC20|102-13139|33.9488611374|-118.3941520089|            10010|
+-------------+---------+-------------

70684

In [28]:
df = routes_with_most_occur_shape.where("route_id like '180%'")
df.show(10)
df.count()

+-------------+---------+-------------+---------------+-----------------+
|     shape_id| route_id| shape_pt_lat|   shape_pt_lon|shape_pt_sequence|
+-------------+---------+-------------+---------------+-----------------+
|1800198_DEC20|180-13139|34.1001914421|-118.3255826782|            10001|
|1800198_DEC20|180-13139|34.1001875827|-118.3255872876|            10002|
|1800198_DEC20|180-13139|34.1001655287|-118.3256136273|            10003|
|1800198_DEC20|180-13139|34.1001104826|-118.3256464449|            10004|
|1800198_DEC20|180-13139|34.1000912298|-118.3256529763|            10005|
|1800198_DEC20|180-13139|34.0997972097|-118.3256518313|            10006|
|1800198_DEC20|180-13139| 34.099799986|-118.3266593125|            10007|
|1800198_DEC20|180-13139|34.1016300729|-118.3266598555|            10008|
|1800198_DEC20|180-13139|34.1016385394|-118.3253551031|            10009|
|1800198_DEC20|180-13139|34.1016385394|-118.3253551031|            20001|
+-------------+---------+-------------

717

In [7]:
shapes_index_from_0 = shapes\
    .select(
        "shape_id",
        struct("shape_pt_lat", "shape_pt_lon").alias("from_latlon"),
        "shape_pt_sequence"
    )\
    .orderBy("shape_id", "shape_pt_sequence")\
    .withColumn(
        "index",
        row_number().over(Window.orderBy(monotonically_increasing_id())) - 1
    )

shapes_index_from_1 = shapes\
    .select(
        "shape_id",
        struct("shape_pt_lat", "shape_pt_lon").alias("to_latlon"),
        "shape_pt_sequence"
    )\
    .orderBy("shape_id", "shape_pt_sequence")\
    .withColumn(
        "index",
        row_number().over(Window.orderBy(monotonically_increasing_id()))
    )

shape_segments = shapes_index_from_0\
    .join(shapes_index_from_1, ["shape_id", "index"])\
    .select(
        "shape_id",
        "from_latlon",
        "to_latlon"
    )\
    .where(is_not_the_same("from_latlon", "to_latlon"))

shape_segments.cache()
shape_segments.show(1)

+------------+--------------------+--------------------+
|    shape_id|         from_latlon|           to_latlon|
+------------+--------------------+--------------------+
|100328_DEC20|{34.0837396369, -...|{34.0837435449, -...|
+------------+--------------------+--------------------+
only showing top 1 row



In [8]:
route_segments = routes\
    .join(trips, "route_id")\
    .select("route_id", "shape_id")\
    .drop_duplicates()\
    .join(shape_segments, "shape_id")\
    .select("route_id", "from_latlon", "to_latlon")

route_segments.cache()
route_segments.show(1)

+---------+--------------------+--------------------+
| route_id|         from_latlon|           to_latlon|
+---------+--------------------+--------------------+
|180-13139|{34.1017731965, -...|{34.1017696859, -...|
+---------+--------------------+--------------------+
only showing top 1 row



In [32]:
stops = spark.read.csv('hdfs://namenode:8020/ola/static_data/stops.txt', header=True)
stops.show(5)

+-------+---------+--------------------+---------+-----------+-----------+--------+-------------+--------------+---------+
|stop_id|stop_code|           stop_name|stop_desc|   stop_lat|   stop_lon|stop_url|location_type|parent_station|tpis_name|
+-------+---------+--------------------+---------+-----------+-----------+--------+-------------+--------------+---------+
|      1|        1| Paramount / Slauson|     null|  33.973248|-118.113113|    null|         null|          null|     null|
|      3|        3|    Jefferson / 10th|     null|  34.025471|-118.328402|    null|         null|          null|     null|
|      6|        6|120th / Augustus ...|     null|  33.924696|-118.242222|    null|         null|          null|     null|
|      7|        7|120th / Martin Lu...|     null|  33.924505|-118.240369|    null|         null|          null|     null|
|     12|       12|   15054 Sherman Way|     null|  34.201075|-118.461953|    null|         null|          null|     null|
+-------+-------

In [33]:
calendar = spark.read.csv('hdfs://namenode:8020/ola/static_data/calendar.txt', header=True)
calendar.show(5)

+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
|          service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
|DEC20-D01CAR-3_Su...|     0|      0|        0|       0|     0|       0|     1|  20201213|20210620|
|DEC20-D01CAR-1_We...|     1|      1|        1|       1|     1|       0|     0|  20201214|20210625|
|DEC20-D01CAR-2_Sa...|     0|      0|        0|       0|     0|       1|     0|  20201219|20210626|
|DEC20-D02CAR-3_Su...|     0|      0|        0|       0|     0|       0|     1|  20201213|20210620|
|DEC20-D02CAR-1_We...|     1|      1|        1|       1|     1|       0|     0|  20201214|20210625|
+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
only showing top 5 rows



In [34]:
calendar_dates = spark.read.csv('hdfs://namenode:8020/ola/static_data/calendar_dates.txt', header=True)
calendar_dates.show(5)

+--------------------+--------+--------------+
|          service_id|    date|exception_type|
+--------------------+--------+--------------+
|DEC20-D01CAR-3_Su...|20201225|             1|
|DEC20-D01CAR-3_Su...|20210101|             1|
|DEC20-D01CAR-3_Su...|20210531|             1|
|DEC20-D01CAR-1_We...|20201225|             2|
|DEC20-D01CAR-1_We...|20210101|             2|
+--------------------+--------+--------------+
only showing top 5 rows



In [35]:
stop_times = spark.read.csv('hdfs://namenode:8020/ola/static_data/stop_times.txt', header=True)
stop_times.show(5)

+--------------------+------------+--------------+-------+-------------+--------------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|       stop_headsign|pickup_type|drop_off_type|
+--------------------+------------+--------------+-------+-------------+--------------------+-----------+-------------+
|52088401-DEC20-D0...|    05:39:00|      05:39:00|  10246|            1|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:40:00|      05:40:00|  10248|            2|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:41:00|      05:41:00|   9371|            3|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:42:00|      05:42:00|   9350|            4|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:43:00|      05:43:00|   9351|            5|611 - Vernon Station|          0|            0|
+--------------------+------------+-----